# Chemical Products Analysis

In this notebook I'm going to look into the Chemical products category from RAPEX. I will use all the alerts from 2005 to 2018.

## Required imports

In [1]:
import requests
import pandas as pd
import codecs
import urllib.parse
import html5lib
from bs4 import BeautifulSoup
import plotly.offline as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
pd.set_option('display.max_colwidth', 300)

## Configuration

In [2]:
category = "Chemical products"
start_year = 2005
end_year = 2018
data_file = 'data/rapex_data.xls'

## Data download

In [3]:
print("Executing search...")
session = requests.Session()
category = urllib.parse.quote_plus(category)
rapex_search_url = "https://ec.europa.eu/consumers/consumers_safety/safety_products/rapex/alerts/?event=search.jsonData&search_year={}&productCategory={}&{}"
sorting = "iSortCol_0=1&sSortDir_0=desc&iSortingCols=1"
years = "%2C".join([str(i) for i in range(start_year, end_year + 1)])
search_url = rapex_search_url.format(years, category, sorting)
session.get(search_url)

print("Downloading Excel data...")
rapex_excel_url = "https://ec.europa.eu/consumers/consumers_safety/safety_products/rapex/alerts/?event=main.search.Excel"
result = session.get(rapex_excel_url)
content = result.text

print("Saving data to {}...".format(data_file))
with codecs.open(data_file, 'w') as f:
    f.write(content)
print("Data download complete.")

Executing search...
Saving data to data/rapex_data.xls...
Data download complete.


## Load the data into a Pandas dataframe

In [4]:
html_table = pd.read_html(content, index_col=False)
df = pd.concat(html_table)
header = df.iloc[0]
df = df[1:]
df.columns = header

df.shape

(547, 23)

### Columns in the dataset

In [5]:
list(df)

['Year',
 'Week',
 'Risk level',
 'Product user',
 'Alert number',
 'Alert submitted by',
 'Category',
 'Product',
 'Brand',
 'Name',
 'Type / number of model',
 'Batch number / Barcode',
 'OECD Portal Category',
 'Description',
 'Country of origin',
 'Counterfeit',
 'Risk type',
 'Technical defect',
 'Risk',
 'Measures adopted by notifying country',
 'Products were found and measures were taken also in',
 'Company recall page',
 'URL of Case']

## Initial Data analysis

### Number of samples per year

Let's start by looking at the distribution of the data by year.

In [6]:
year_counts = df['Year'].value_counts().rename_axis('Year').reset_index(name='Count')
year_counts.sort_values(by=['Year'])

,Year,Count
13,2005,5
12,2006,15
11,2007,18
10,2008,21
6,2009,44
9,2010,29
8,2011,38
1,2012,57
2,2013,56
0,2014,64


Most samples are from the year 2014.

In [7]:
year_counts_plot = [go.Bar(x=year_counts['Year'], y=year_counts['Count'])]
py.iplot({'data': year_counts_plot,
                  'layout': {
            'title': 'Data distribution by Year',
            'xaxis': {
                 'title': 'Year'},
            'yaxis': {
                'title': 'Count '}
        }})

### Product distribution

An important insight is to know which products are represented the most in the dataset.

In [8]:
df['Product'] = df['Product'].str.lower()
product_counts = df['Product'].value_counts().rename_axis('Product').reset_index(name='Count')
product_counts.sort_values(by=['Count'], ascending=False)

print("The most represented products are:")
product_counts.head(10)

The most represented products are:


,Product,Count
0,tattoo ink,169
1,glue,44
2,liquid for e-cigarettes,24
3,valve oil for musical instruments,16
4,tattoo or permanent make-up ink,12
5,super glue,9
6,rubber solution in bicycle repair kit,7
7,liquid for electronic cigarettes,6
8,bicycle tyre repair kit,6
9,poppers,5


### Merge similar product categories

In [9]:
df['PCategory'] = "other"
df.loc[df['Product'].str.contains('tattoo|permanent\smake-up|permanent\smake\sup', na=False),'PCategory'] = "tatoo"
df.loc[df['Product'].str.contains('glue|fixative', na=False),'PCategory'] = "glue"
df.loc[df['Product'].str.contains('e-cigarette|electronic\scigar', na=False),'PCategory'] = "e-cigarette"
df.loc[df['Product'].str.contains('cleaner|dissolvent|wash|antibacterial|cleaning|detergent|bleach|stain\sremover|paint\sremover|decalcifier|thinner|descaling|paint\sstripper', na=False),'PCategory'] = "cleaning"
df.loc[df['Product'].str.contains('bicycle|car|rubber|tyre|tire|wheel|antifreeze|windscreen|windshield|vehicle|bike', na=False),'PCategory'] = "bicycle and car"
df.loc[df['Product'].str.contains('freshener|aerosol|spray|perfume|incense|fragrance|scent|aromas', na=False),'PCategory'] = "aerosol"
df.loc[df['Product'].str.contains('oil|fuel', na=False),'PCategory'] = "oil"
df.loc[df['Product'].str.contains('popper', na=False),'PCategory'] = "poppers"

category_counts = df['PCategory'].value_counts().rename_axis('PCategory').reset_index(name='Count').sort_values(by=['Count'], ascending=False)
category_counts

,PCategory,Count
0,tatoo,216
1,glue,88
2,bicycle and car,62
3,other,45
4,e-cigarette,37
5,cleaning,36
6,aerosol,33
7,oil,23
8,poppers,7


### Plot the distribution

In [10]:
product_counts_plot = [go.Bar(x=category_counts['PCategory'], y=category_counts['Count'])]
py.iplot({'data': product_counts_plot,
                  'layout': {
            'title': 'Data distribution by Product',
            'xaxis': {
                 'title': 'Product'},
            'yaxis': {
                'title': 'Count '}
        }})

### Risk level

In [11]:
df['Risk level'].value_counts().rename_axis('Risk').reset_index(name='Count')

,Risk,Count
0,Serious risk,503
1,Other risk level,44


## Cancer Risk

### How many risk alerts are related to cancer?

In [12]:
df['Cancer'] = 0
df['Risk'] = df['Risk'].str.lower()
df.loc[df["Risk"].str.contains('cancer'), 'Cancer'] = 1
df.loc[df["Risk"].str.contains('carcinogen'), 'Cancer'] = 1
df.loc[df["Risk"].str.contains('carcinogenic'), 'Cancer'] = 1
cancer_counts = df['Cancer'].value_counts()
cancer_counts

0    363
1    184
Name: Cancer, dtype: int64

In [13]:
df['Cancer'].value_counts(normalize = True)

0    0.66362
1    0.33638
Name: Cancer, dtype: float64

In [14]:
labels = ['Cancer Related','Other']
values = [cancer_counts[1],cancer_counts[0]]
trace = go.Pie(labels=labels, values=values)
py.iplot([trace], filename='basic_pie_chart')

Around 1/3 of the risks are cancer related.

### Which product categories contain risk of cancer

In [15]:
products_cancer = df.loc[df['Cancer']==1]
products_cancer['PCategory'].value_counts().rename_axis('PCategory').reset_index(name='Count').sort_values(by=['Count'], ascending=False)

,PCategory,Count
0,tatoo,134
1,bicycle and car,20
2,glue,9
3,other,9
4,aerosol,6
5,poppers,6


### Plot of the distribution of cancer risk among individual categories

In [16]:
tatoo_products = df.loc[df['PCategory']=='tatoo','Cancer'].value_counts()
glue_products = df.loc[df['PCategory']=='glue','Cancer'].value_counts()
bicycle_products = df.loc[df['PCategory']=='bicycle and car','Cancer'].value_counts()
aerosol_products = df.loc[df['PCategory']=='aerosol','Cancer'].value_counts()
poppers_products = df.loc[df['PCategory']=='poppers','Cancer'].value_counts()
other_products = df.loc[df['PCategory']=='other','Cancer'].value_counts()

cols = ['tatoo', 'glue', 'bicycle and car', 'aerosol', 'poppers', 'other']
y1 = [tatoo_products[1], glue_products[1], bicycle_products[1], 
    aerosol_products[1], poppers_products[1], other_products[1]]
y2 = [tatoo_products[0], glue_products[0], bicycle_products[0], 
    aerosol_products[0], poppers_products[0], other_products[0]]

trace1 = go.Bar(x=cols, y=y1, name='Cancer')
trace2 = go.Bar(x=cols, y=y2, name='Other Risks')
data = [trace1, trace2]
layout = go.Layout(barmode='stack')
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='stacked-bar')

### Summary

In [17]:
results_df = pd.DataFrame.from_dict({
    "PCategory": cols,
    "Cancer risk": y1,
    "Other risks": y2
})

results_df['Sum'] = results_df['Cancer risk'] + results_df['Other risks']
results_df['Cancer risk %'] = (results_df['Cancer risk']/results_df['Sum'])*100

print(results_df)

         PCategory  Cancer risk  Other risks  Sum  Cancer risk %
0            tatoo          134           82  216      62.037037
1             glue            9           79   88      10.227273
2  bicycle and car           20           42   62      32.258065
3          aerosol            6           27   33      18.181818
4          poppers            6            1    7      85.714286
5            other            9           36   45      20.000000


Conclusion most cancer related alerts were products from the tatoo category.

### Why there are no e-cigarette related cancer risks?

In [18]:
e_cig_risks = df.loc[df['PCategory']=='e-cigarette','Risk']
e_cig_risks.str.contains('carcinogen|cancer').value_counts()

False    37
Name: Risk, dtype: int64

### What are some of the risks of e-cigarettes

#### Related to packaging, label or warning

In [19]:
e_cig_risks.str.contains('label|packaging|safety\sadvice|danger\swarning').value_counts()

True     33
False     4
Name: Risk, dtype: int64

#### Related specifically to nicotin

In [20]:
e_cig_risks.str.contains('nicotin').value_counts()

True     31
False     6
Name: Risk, dtype: int64

Conclusion: Because e-cigarettes deliver nicotine without the tar and many of the other cancer-linked chemicals found in tobacco, they’re thought to pose less of a cancer risk. 

However most cases are related to wrong or missing labelling or missing information about nicotine amounts.

### Which bicycle and car products are related to cancer risk?

In [21]:
bc_products = df.loc[df['PCategory']=='bicycle and car']
bc_products = bc_products.loc[bc_products['Cancer']==1]

bc_product_counts = bc_products['Product'].value_counts().rename_axis('Product').reset_index(name='Count')
bc_product_counts.sort_values(by=['Count'], ascending=False)
bc_product_counts

,Product,Count
0,rubber solution in bicycle repair kit,7
1,bicycle repair kit,2
2,rubber solution in a bicycle repair kit,2
3,"rubber solution in bike repair set containing rubber solution, spanner and several plasters, 'fahrrad flick' set",1
4,bicycle repair set with rubber cement red sun,1
5,pax rubber cement,1
6,bicycle tyre repair kit - rubber solution,1
7,bicycle repair kit with glue,1
8,bicycle tyre repair kit,1
9,bicycle repair kit - rubber solution,1


Looks like the cancer risk is mainly due to benzene present in bicycle tyre repair kits.

In [22]:
bc_products.loc[bc_products['Product'].str.contains('bicycle.*repair.*kit'), 'Product'] = 'bicycle repair kit'
bc_product_counts = bc_products['Product'].value_counts().rename_axis('Product').reset_index(name='Count')
bc_product_counts.sort_values(by=['Count'], ascending=False)
bc_product_counts

,Product,Count
0,bicycle repair kit,16
1,"rubber solution in bike repair set containing rubber solution, spanner and several plasters, 'fahrrad flick' set",1
2,tyre repair kit,1
3,pax rubber cement,1
4,bicycle repair set with rubber cement red sun,1


Conclusion: Most cancer related alerts from the bicycle and car category are due to chemicals in the bicycle repair kits.

## Countries

'Alert submitted by' contains the name of the country where the risk was submitted

In [23]:
df['Alert submitted by'].value_counts().rename_axis('user').reset_index(name='Count')

,user,Count
0,Germany,123
1,Italy,104
2,France,56
3,The Netherlands,55
4,Spain,41
5,Sweden,33
6,Lithuania,26
7,Cyprus,19
8,United Kingdom,16
9,Denmark,14


### Countries of origin

In [24]:
df['Country of origin'].value_counts().rename_axis('user').reset_index(name='Count')

,user,Count
0,United States,173
1,China,136
2,Unknown,46
3,Germany,26
4,Italy,24
5,United Kingdom,20
6,France,14
7,Poland,12
8,The Netherlands,11
9,Taiwan,9
